Array traps and optimisation
============================

First let's start 3 workers

In [ ]:
import dask.array as da
from dask.distributed import Client, progress
client = Client(
    processes=False,
    n_workers=3,
    threads_per_worker=1,
)
client

Chunk size
----------

- Too large chunks don't split work efficiently.
- Too small and too much time is lost in communication and other overheads.
- 100Mb~1Gb per chunks is usually good. Scheduling a single task takes around ~1ms.

In [ ]:
print("1 chunk")
arr = da.random.random((6000, 6000), chunks=(6000, 6000))
%time x = arr.sum().compute()
print("3 chunks")
arr = da.random.random((6000, 6000), chunks=(2000, 6000))
%time x = arr.sum().compute()
print("4 chunks")
arr = da.random.random((6000, 6000), chunks=(3000, 3000))
%time x = arr.sum().compute()
print("36 chunks")
arr = da.random.random((6000, 6000), chunks=(1000, 1000))
%time x = arr.sum().compute()
print("400 chunks")
arr = da.random.random((6000, 6000), chunks=(300, 300))
%time x = arr.sum().compute()
print("auto")
arr = da.random.random((6000, 6000), chunks="auto")
%time x = arr.sum().compute()

In [ ]:
arr

Operation order
---------------

Dask has a symbolic tree of operations, but little tools for optimization.  
It does not reorder operations for faster computation:

In [ ]:
A = da.random.random((3000, 3000), chunks=(1000, 1000))
B = da.random.random((3000, 3000), chunks=(1000, 1000))
v = da.random.random((3000, 1), chunks=(1000, 1))
MM1 = (A @ B) @ v
MM2 = A @ (B @ v)
%time MM1.compute()
%time MM2.compute()

In [ ]:
# einsum can do operation in the right order, but the operation is not optimized
einMM = da.einsum("ij,jk,kl->il", A, B, v)
%time einMM.compute()

Chunks alignment
----------------
Operation between array of any chunks will work.  
However, when chunks are not matching, dask must re-chunks the arrays before doing the operation.  
This slows down the computation significantly.

In [ ]:
A = da.random.random((3000, 3000), chunks=(750, 750))
B = da.random.random((3000, 3000), chunks=(600, 600))

print("Same chunks")
%time A + A
%time B + B

print("Mixed chunks")
%time A + B

Usage with other packages
-------------------------
Function passed to `map_block` or `delayed` can be optimized with other tools!  

In [ ]:
from dask import delayed
import numba
import numpy as np

@numba.jit
def convolve2d(a):
    return a[2:, 1:-1] + a[:-2, 1:-1] + a[1:-1, 2:] + a[1:-1, :-2] - 4 * a[1:-1, 1:-1]

f = delayed(convolve2d)(np.ones((10, 10)))
f.compute()

In [ ]:
x = da.from_array(np.arange(100).reshape((10, 10)), chunks=(10, 10))
out = da.overlap.map_overlap(convolve2d, x, depth=(1, 1), boundary=("reflect", "reflect"))
out.compute()